# More data cleaning

The Ravelry data has columns where each row value is a dictionary. To make things simpler, I'd rather break that information up before analysis than try to analyze while dealing with the dictionary structure.


In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
import ast


In [ ]:
patterns_df = pd.read_csv('../data/df_patterns_clean.csv', low_memory = False)

In [ ]:
pd.set_option('display.max_columns', 60)
patterns_df.head()

Pattern df cleaning to do:
- created_at : truncate to year/month/day, possibly convert format
- standardize yarn_weight_description (perhaps group thread and cobweb, aran/worsted into one or the other, dk/sport the same - look at pattern guauge to determine which way to go)
- packs : break out called-for yarn
- craft : break out craft name to new column
- pattern_type : break out clothing(T/F) and type name (Socks, Shawl/Wrap etc)


Wait on:
- pattern_needle_sizes : see if I need this info for analysis
- yarn_weight : might not need at all
- pattern_categories : might not need since pattern_type contains top-level category
- pattern_attributes : if I get extraordinarily ambitious, don't think I'm going to have time to get this far in the weeds

Thoughts on dealing with the dictionary columns:

As an example, the called-for yarn in the pattern details df is a key/value in a larger dictionary column. I want to extract the yarn name on its own into a new column.

In [ ]:
patterns_df = patterns_df.rename(columns = {'name' : 'patt_name'})

In [ ]:
patterns_df.pattern_type.values

In [ ]:
patterns_df.craft.value_counts()

# Split faux-dictionary columns

In [ ]:
# split craft column

patterns_df['craft'] = patterns_df['craft'].apply(lambda x : dict(eval(x)))
temp = patterns_df['craft'].apply(pd.Series)
patterns_df = pd.concat([patterns_df, temp], axis = 1).drop('craft', axis = 1)
patterns_df.head()


In [ ]:
patterns_df = patterns_df.drop(['id', 'permalink'], 1)
patterns_df = patterns_df.rename(columns = {'name' : 'craft_name'})
patterns_df.head()

In [ ]:
print(patterns_df.pattern_type.values)

In [ ]:
# tried same code on other problem columns, but getting errors even with other approaches
# new approach - they're strings, so use split and then trim extra characters

pattern_type_df = patterns_df.pattern_type.str.split(", ", expand = True)
pattern_type_df

In [ ]:
# rename columns I want to keep
pattern_type_df = pattern_type_df.rename(columns = {0 : 'clothing', 2 : 'type_name'})

# remove label part of clothing column
pattern_type_df['clothing'] = pattern_type_df['clothing'].str.replace(r"{'clothing': ", '')

# remove label part of type_name column
pattern_type_df['type_name'] = pattern_type_df['type_name'].str.replace(r"'name': '", '')

# slice off trailing character in type_name column
pattern_type_df['type_name'] = pattern_type_df['type_name'].str.slice(0, -1)

# drop extra columns
pattern_type_df = pattern_type_df.drop([1, 3], 1)
pattern_type_df.head()


In [ ]:
# result as expected
pattern_type_df.clothing.value_counts()

In [ ]:
# result as expected
pattern_type_df.type_name.value_counts()

In [ ]:
# merge back to patterns_df on index
patterns_df = patterns_df.merge(pattern_type_df, how = 'outer', left_index = True, right_index = True)
patterns_df.tail()

In [ ]:
# drop original column
patterns_df = patterns_df.drop(['pattern_type'], 1)
patterns_df.tail()

In [ ]:
# repeat process for packs column
packs_df = patterns_df.packs.str.split(", ", expand = True)
packs_df.head()

In [ ]:
print(packs_df[10].values)

In [ ]:
# rename columns I want to keep
packs_df = packs_df.rename(columns = {10 : 'patt_yarn', 16 : 'patt_yarn_weight'})

# remove label part of patt_yarn column
packs_df['patt_yarn'] = packs_df['patt_yarn'].str.replace(r"'yarn_name': '", '')

# remove label part of patt_yarn_weight column
packs_df['patt_yarn_weight'] = packs_df['patt_yarn_weight'].str.replace(r"'name': '", '')

# slice off trailing characters in patt_yarn and Patt_yarn_weight columns
packs_df['patt_yarn'] = packs_df['patt_yarn'].str.slice(0, -1)
packs_df['patt_yarn_weight'] = packs_df['patt_yarn_weight'].str.slice(0, -1)

# drop extra columns
packs_df = packs_df.drop(packs_df.iloc[:, 0:10], axis = 1)
packs_df.head()


In [ ]:
# drop extra columns part 2

packs_df = packs_df.drop(packs_df.iloc[:, 1:6], axis = 1)
packs_df.head()

In [ ]:
# final dropping of extra columns

packs_df = packs_df.drop(packs_df.iloc[:, 2:], axis = 1)
packs_df.head()

In [ ]:
# merge back to patterns_df on index
patterns_df = patterns_df.merge(packs_df, how = 'outer', left_index = True, right_index = True)

# drop original column
patterns_df = patterns_df.drop(['packs'], 1)
patterns_df.tail()

# Other cleaning on patterns_df

In [ ]:
patterns_df.info()

In [ ]:
# truncate created_at column
patterns_df['created_at'] = patterns_df['created_at'].str.slice(0, 11)
patterns_df.tail()

In [ ]:
patterns_df.to_csv('../data/df_patterns_clean2.csv', index = False)

----------------

# Shop data cleaning

In [ ]:
shops_df = pd.read_csv('../data/df_shop_clean.csv')

In [ ]:
shops_df.head()

In [ ]:
# slice off extraneous info from country and state columns

shops_df['country'] = shops_df['country'].str.replace(r"{'id': 229, 'name': '", '')
shops_df['country'] = shops_df['country'].str.slice(0, -2)
shops_df['state'] = shops_df['state'].str.slice(22, -2)

shops_df.tail()

In [ ]:
# shops with no zip or lat/long

noloc_df = shops_df[(shops_df['latitude'].isna()) &
                    (shops_df['longitude'].isna())]

noloc_df.info()

In [ ]:
noloc_df.head()

In [ ]:
shops_df = shops_df[~shops_df['latitude'].isna() & ~shops_df['longitude'].isna()]

In [ ]:
shops_df.head()

In [ ]:
shops_df = shops_df.drop_duplicates(['latitude', 'longitude'])
shops_df.info()

In [ ]:
shops_df = shops_df.reset_index(drop = True)
shops_df.info()

In [ ]:
shops_df.to_csv('../data/df_shops_clean2.csv', index = False)

# Yarn data cleaning

In [ ]:
yarns_df = pd.read_csv('../data/df_yarn_clean.csv')
yarns_df.head()

Cleaning needed:
- texture : clean up categories (multiple versions of plied etc)
- break out yarn_weight, also drop the extra one that made it through the previous step
- break out yarn_fibers

save for if needed:
- break out min and max needle size, keep metric
- break out min and max hook size, keep metric

In [ ]:
yarns_df = yarns_df.rename(columns = {'name' : 'yarn_name'})

In [ ]:
yarns_df.texture.value_counts()

In [ ]:
# trim white space
yarns_df['texture'] = yarns_df['texture'].str.strip()
yarns_df.texture.value_counts()

Easier to do the rest of this step in Excel.

# Split faux-dictionary columns

In [ ]:
# yarn_weight
yarn_weight_df = yarns_df.yarn_weight_x.str.split(", ", expand = True)
yarn_weight_df

In [ ]:
# rename columns I want to keep
yarn_weight_df = yarn_weight_df.rename(columns = {5 : 'yarn_weight', 7 : 'wpi'})

# remove label part of columns
yarn_weight_df['yarn_weight'] = yarn_weight_df['yarn_weight'].str.replace(r"'name': '", '')
yarn_weight_df['wpi'] = yarn_weight_df['wpi'].str.replace(r"'wpi': '", '')

# slice off trailing character in columns
yarn_weight_df['yarn_weight'] = yarn_weight_df['yarn_weight'].str.slice(0, -1)
yarn_weight_df['wpi'] = yarn_weight_df['wpi'].str.slice(0, -2)

# drop extra columns
yarn_weight_df = yarn_weight_df.drop([0, 1, 2, 3, 4, 6], 1)
yarn_weight_df.head()


In [ ]:
# merge back to yarns_df on index
yarns_df = yarns_df.merge(yarn_weight_df, how = 'outer', left_index = True, right_index = True)
yarns_df.tail()

In [ ]:
# drop original columns
yarns_df = yarns_df.drop(['yarn_weight_x', 'yarn_weight_y'], 1)
yarns_df.tail()

In [ ]:
# yarn_fibers
yarn_fibers_df = yarns_df.yarn_fibers.str.split(", ", expand = True)
yarn_fibers_df.head()

In [ ]:
# limit to info on top 3 fibers per yarn
yarn_fibers_df[39].value_counts()

In [ ]:
# drop extra columns from the end
yarn_fibers_df = yarn_fibers_df.drop(yarn_fibers_df.iloc[:, 39:], axis = 1)
yarn_fibers_df.head()

In [ ]:
yarn_fibers_df.tail()

In [ ]:
yarn_fibers_df[21].value_counts()

Getting info out of yarn_fibers isn't going to work without turning it into a real dictionary first. (Each fiber in a yarn can have up to 13 attributes, but not all have the same number so just spliting into columns to deal with it won't work.) Tried code that worked for the craft column in patterns_df but getting same error as before with other columns.

yarns_df['yarn_fibers'] = yarns_df['yarn_fibers'].apply(lambda x : dict(eval(x)))
temp = yarns_df['yarn_fibers'].apply(pd.Series)
yarns_df = pd.concat([yarns_df, temp], axis = 1).drop('yarn_fibers', axis = 1)
yarns_df.head()

Found this solution of stackoverflow (https://stackoverflow.com/questions/38231591/splitting-dictionary-list-inside-a-pandas-column-into-separate-columns), but it hasn't worked either.

yarn_fiber_df = pd.json_normalize(yarns_df['yarn_fibers'])

I think json_normalize might work if the column was already a true dictionary (nope), but that's the step at which every approach goes wrong.

Tried turning dictionary into a dataframe, but the arrays are different lengths.

In [ ]:
yarns_df['yarn_fibers'].loc[2]

In [ ]:
ast.literal_eval(yarns_df['yarn_fibers'].loc[2])

In [ ]:
type(ast.literal_eval(yarns_df['yarn_fibers'].loc[2]))

In [ ]:
#thought this might work, but it doesn't do quite what I want

#yarns_df['yarn_fibers'] = yarns_df['yarn_fibers'].apply(lambda x : ast.literal_eval(x))
#temp = yarns_df['yarn_fibers'].apply(pd.Series)
#yarns_df = pd.concat([yarns_df, temp], axis = 1)#.drop('yarn_fibers', axis = 1)
#yarns_df.head()

In [ ]:
yarns_df['yarn_fibers'].apply(lambda x : len(ast.literal_eval(x))).value_counts()
# if I understand correctly, this result indicates how many fibers are listed for each yarn

In [ ]:
yarn_dict = yarns_df['yarn_fibers'].apply(lambda x : ast.literal_eval(x)[0] if len(ast.literal_eval(x)) > 0 else {})
# properly turns column into a dictionary
# does it only pull the first fiber? I think so. How to change this to pull each yarn firber component in turn?
# I think this is what Mahesh was talking about when he recommended a function.

In [ ]:
yarnfiber_df = yarn_dict.apply(pd.Series)
# takes each dictionary element and puts it in a column

In [ ]:
yarnfiber_df
# fiber_type and fiber_category are both dictionaries so they need further steps

In [ ]:
#apply_test2 = temp['fiber_type'].apply(lambda x : ast.literal_eval(x) if len(ast.literal_eval(x)) > 0 else {})
yarnfiber_type_df = yarnfiber_df['fiber_type'].apply(pd.Series)
yarnfiber_type_df
# repeat .apply to breakout formerly nested dictionaries

Mahesh recommends a function to accomplish turning problem columns into dictionaries and then breaking them into constituent parts. Also, use the function in place of lambda.

What are the steps?


In [ ]:
yarnfiber_category_df = yarnfiber_df['fiber_category'].apply(pd.Series)
yarnfiber_category_df

In [ ]:
yarnfiber_parent_df = yarnfiber_category_df['parent'].apply(pd.Series)
yarnfiber_parent_df

Going to take the first fiber listed for purpose of immediate analysis.

In [ ]:
# merge percentage from yarnfiber_df
yarns_df = yarns_df.join(yarnfiber_df, rsuffix='_y')
yarns_df = yarns_df.drop(['id', 'fiber_type', 'fiber_category'], axis = 1)
yarns_df.head()

In [ ]:
# merge fiber info from yarnfiber_type_df
yarns_df = yarns_df.join(yarnfiber_type_df)
yarns_df = yarns_df.drop([0, 'id'], axis = 1)
yarns_df.head()

In [ ]:
yarns_df = yarns_df.drop([0, 'id'], axis = 1)
yarns_df.head()

In [ ]:
# merge fiber category info from yarnfiber_category_df
yarns_df = yarns_df.join(yarnfiber_category_df, rsuffix='_y')
yarns_df = yarns_df.drop([0, 'id', 'parent', 'permalink'], axis = 1)
yarns_df.head()


In [ ]:
yarns_df = yarns_df.rename(columns = {'name_y' : 'fiber_catt_name'})
yarns_df.head()

In [ ]:
# save cleaned yarn dataframe to csv

yarns_df.to_csv('../data/df_yarn_clean2.csv', index = False)